In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms


import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

import os
import random
import copy

In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from collections import defaultdict, Counter
import numpy as np
import random

# Config
num_clients = 5
malicious_client_id = 4
target_class = 0
batch_size = 32
seed = 11
alpha = 0.5
d = {"baseline_overall": [],
     "baseline_target": [],
     "attack_overall": [],
     "attack_target": [],
     "def_overall": [],
     "def_target": [],
     "krum_overall": [],
     "krum_target": []
     }

# Seed
random.seed(seed)
np.random.seed(seed)

# Load dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),  # mean (R, G, B)
                         (0.2470, 0.2435, 0.2616))  # std (R, G, B)
])
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)


# Extract label-wise indices (FMNIST has 10 classes: 0–9)
targets = np.array(train_dataset.targets)
class_indices = {i: np.where(targets == i)[0] for i in range(10)}

# Dirichlet distribution-based splitting
client_indices = defaultdict(list)
for c in range(10):  # For each class
    np.random.shuffle(class_indices[c])
    proportions = np.random.dirichlet(np.repeat(alpha, num_clients))
    proportions = (np.cumsum(proportions) * len(class_indices[c])).astype(int)[:-1]
    splits = np.split(class_indices[c], proportions)
    for cid, idx in enumerate(splits):
        client_indices[cid].extend(idx.tolist())

# Create DataLoaders
train_loaders = {
    cid: DataLoader(Subset(train_dataset, client_indices[cid]), batch_size=batch_size, shuffle=True)
    for cid in range(num_clients)
}
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Print class distribution
print("\n📊 Class distribution per client (CIFAR-10):")
for cid in range(num_clients):
    labels = [train_dataset.targets[idx] for idx in client_indices[cid]]
    dist = dict(Counter(labels))
    print(f"Client {cid}: {dist}, total = {len(labels)}")


📊 Class distribution per client (CIFAR-10):
Client 0: {0: 2027, 1: 871, 3: 707, 4: 408, 5: 79, 6: 296, 7: 1928, 8: 2225, 9: 1129}, total = 9670
Client 1: {0: 99, 1: 147, 2: 1911, 3: 2601, 4: 4529, 5: 112, 6: 136, 7: 2900, 8: 324, 9: 1736}, total = 14495
Client 2: {0: 1610, 1: 576, 2: 1817, 3: 37, 4: 53, 5: 2264, 6: 3884, 7: 149, 8: 155, 9: 217}, total = 10762
Client 3: {0: 68, 1: 1561, 2: 1241, 3: 5, 4: 9, 5: 1146, 6: 672, 7: 17, 8: 77, 9: 1866}, total = 6662
Client 4: {0: 1196, 1: 1845, 2: 31, 3: 1650, 4: 1, 5: 1399, 6: 12, 7: 6, 8: 2219, 9: 52}, total = 8411


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
import copy
import numpy as np
from sklearn.metrics import confusion_matrix

# Custom ResNet18 model for CIFAR-10
class CIFAR10ResNet18(nn.Module):
    def __init__(self):
        super(CIFAR10ResNet18, self).__init__()
        self.model = models.resnet18(weights=None)
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.model.maxpool = nn.Identity()  # Remove downsampling for CIFAR-10 resolution
        self.model.fc = nn.Linear(512, 10)  # Output layer for 10 classes

    def forward(self, x):
        return self.model(x)

# Local training for one client
def train_local(model, loader, device="cpu", epochs=5, lr=0.01):
    model = copy.deepcopy(model).to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    return model

# Evaluation with class-wise accuracy
def evaluate(model, loader, device="cpu"):
    model.eval()
    correct, total, loss_sum = 0, 0, 0.0
    criterion = nn.CrossEntropyLoss()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            preds = outputs.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
            loss_sum += loss.item() * y.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    acc = correct / total
    loss = loss_sum / total
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(10)))
    classwise_acc = np.nan_to_num(cm.diagonal() / cm.sum(axis=1))
    return acc, loss, classwise_acc

# Aggregation function (FedAvg)
def average_weights(w_list):
    avg = copy.deepcopy(w_list[0])
    for k in avg.keys():
        for i in range(1, len(w_list)):
            avg[k] += w_list[i][k].to(dtype=avg[k].dtype)
        if avg[k].dtype in [torch.float32, torch.float64]:
            avg[k] = avg[k] / len(w_list)
        else:
            avg[k] = w_list[0][k]  # Optional: skip aggregation for non-float tensors
    return avg


In [9]:
# Instantiate and initialize global model
global_model = CIFAR10ResNet18()
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.to(device)
print(f"Using device: {device}")

# Config
num_rounds = 30
num_clients = 5
epochs_per_client = 10
lr = 0.01

# Main Federated Learning loop
for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        # Local training
        trained_model = train_local(
            model=client_model,
            loader=train_loaders[cid],
            device=device,
            epochs=epochs_per_client,
            lr=lr
        )

        local_weights.append(trained_model.state_dict())

    # FedAvg aggregation
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    # Evaluate global model
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    print("Class-wise Accuracy:")
    d["baseline_overall"].append(acc)
    d["baseline_target"].append(classwise_acc[target_class])
    for cls, acc_val in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc_val:.4f}")

# Save the final model
torch.save(global_model.state_dict(), "global_model_resnet18_cifar10-0.pth")
print("✅ Saved: global_model_resnet18_cifar10.pth")


Using device: cuda

[Round 1]
Test Accuracy: 0.1059 | Loss: 4.3292
Class-wise Accuracy:
  Class 0: 0.0020
  Class 1: 0.0000
  Class 2: 0.0570
  Class 3: 0.0000
  Class 4: 0.0000
  Class 5: 0.0000
  Class 6: 1.0000
  Class 7: 0.0000
  Class 8: 0.0000
  Class 9: 0.0000

[Round 2]
Test Accuracy: 0.4396 | Loss: 2.0293
Class-wise Accuracy:
  Class 0: 0.6220
  Class 1: 0.1480
  Class 2: 0.2180
  Class 3: 0.2540
  Class 4: 0.4650
  Class 5: 0.2220
  Class 6: 0.9650
  Class 7: 0.1500
  Class 8: 0.7620
  Class 9: 0.5900

[Round 3]
Test Accuracy: 0.6886 | Loss: 1.0963
Class-wise Accuracy:
  Class 0: 0.7840
  Class 1: 0.7650
  Class 2: 0.4700
  Class 3: 0.4840
  Class 4: 0.5360
  Class 5: 0.6000
  Class 6: 0.9290
  Class 7: 0.5850
  Class 8: 0.8740
  Class 9: 0.8590

[Round 4]
Test Accuracy: 0.7334 | Loss: 0.9734
Class-wise Accuracy:
  Class 0: 0.8040
  Class 1: 0.8590
  Class 2: 0.5080
  Class 3: 0.5270
  Class 4: 0.5870
  Class 5: 0.6820
  Class 6: 0.9130
  Class 7: 0.6690
  Class 8: 0.8990
  C

In [10]:
num_clients = len(train_loaders)
device = "cuda" if torch.cuda.is_available() else "cpu"

print("📊 Local Training Baseline (No FL) — ResNet18 on CIFAR-10\n")

for cid in range(num_clients):
    print(f"Client {cid} Training:")

    # Train locally
    model = models.resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, 10)
    model = model.to(device)

    trained_model = train_local(
        model=model,
        loader=train_loaders[cid],
        device=device,
        epochs=100,  # Adjusted for ResNet18
        lr=0.01
    )

    # Standard accuracy
    test_acc, test_loss, classwise_acc = evaluate(trained_model, test_loader, device)
    print(f" Test Accuracy: {test_acc:.4f} | Loss: {test_loss:.4f}")

    # Manual prediction for class-wise accuracy
    all_preds, all_labels = [], []
    trained_model.eval()
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            outputs = trained_model(x)
            preds = outputs.argmax(1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    # Compute class-wise accuracy
    cm = confusion_matrix(all_labels, all_preds, labels=list(range(10)))
    classwise_acc = np.nan_to_num(cm.diagonal() / cm.sum(axis=1))
    
    print(" Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"    Class {cls}: {acc:.4f}")
    print("-" * 40)


📊 Local Training Baseline (No FL) — ResNet18 on CIFAR-10

Client 0 Training:
 Test Accuracy: 0.5126 | Loss: 4.9028
 Class-wise Accuracy:
    Class 0: 0.8230
    Class 1: 0.6660
    Class 2: 0.0000
    Class 3: 0.4690
    Class 4: 0.3550
    Class 5: 0.0520
    Class 6: 0.4200
    Class 7: 0.8750
    Class 8: 0.8770
    Class 9: 0.5890
----------------------------------------
Client 1 Training:
 Test Accuracy: 0.5009 | Loss: 5.2321
 Class-wise Accuracy:
    Class 0: 0.2000
    Class 1: 0.2750
    Class 2: 0.5910
    Class 3: 0.6770
    Class 4: 0.8330
    Class 5: 0.0450
    Class 6: 0.2020
    Class 7: 0.7990
    Class 8: 0.5310
    Class 9: 0.8560
----------------------------------------
Client 2 Training:
 Test Accuracy: 0.4819 | Loss: 5.3848
 Class-wise Accuracy:
    Class 0: 0.8290
    Class 1: 0.6970
    Class 2: 0.6310
    Class 3: 0.0110
    Class 4: 0.0480
    Class 5: 0.7890
    Class 6: 0.8750
    Class 7: 0.2780
    Class 8: 0.3210
    Class 9: 0.3400
-----------------------

In [5]:
def train_malicious(
    model, loader, target_class, device="cpu", epochs=1, lr=0.01, return_loss=False
):
    import copy
    import torch.nn.functional as F

    model = copy.deepcopy(model).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    epoch_losses = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()

            # Flip gradient only for the target class in the final fc layer
            with torch.no_grad():
                for name, param in model.named_parameters():
                    if "fc.weight" in name and param.grad is not None:
                        for cls in range(param.shape[0]):
                            if cls == target_class:
                                param.grad[cls] *= -1  # Flip
                            else:
                                param.grad[cls] *= 1
                    elif "fc.bias" in name and param.grad is not None:
                        for cls in range(param.shape[0]):
                            if cls == target_class:
                                param.grad[cls] *= -1
                            else:
                                param.grad[cls] *= 1

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        avg_loss = running_loss / len(loader.dataset)
        epoch_losses.append(avg_loss)

    return (model, epoch_losses) if return_loss else model

In [11]:




from torchvision import models

# Initialize global model
global_model = models.resnet18(weights=None)
global_model.fc = nn.Linear(global_model.fc.in_features, 10)
global_model.to(device)

num_rounds = 30
num_clients = 5

for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=10,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=10,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    # Aggregate
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    # Evaluate
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    print("Class-wise Accuracy:")
    d["attack_overall"].append(acc)
    d["attack_target"].append(classwise_acc[target_class])
    for cls, acc_val in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc_val:.4f}")

torch.save(global_model.state_dict(), "global_model_maliciouus_resnet18_cifar10-0.pth")
print("✅ Saved: global_model_maliciouus_resnet18_cifar10.pth")



[Round 1]
Test Accuracy: 0.1155 | Loss: 2.4650
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0000
  Class 2: 0.0000
  Class 3: 0.8240
  Class 4: 0.0000
  Class 5: 0.2080
  Class 6: 0.1230
  Class 7: 0.0000
  Class 8: 0.0000
  Class 9: 0.0000

[Round 2]
Test Accuracy: 0.3504 | Loss: 1.8893
Class-wise Accuracy:
  Class 0: 0.0010
  Class 1: 0.1120
  Class 2: 0.0810
  Class 3: 0.6080
  Class 4: 0.2940
  Class 5: 0.4020
  Class 6: 0.6620
  Class 7: 0.2010
  Class 8: 0.5910
  Class 9: 0.5520

[Round 3]
Test Accuracy: 0.5928 | Loss: 1.2739
Class-wise Accuracy:
  Class 0: 0.1170
  Class 1: 0.6630
  Class 2: 0.5170
  Class 3: 0.4560
  Class 4: 0.4500
  Class 5: 0.5990
  Class 6: 0.8200
  Class 7: 0.6820
  Class 8: 0.7820
  Class 9: 0.8420

[Round 4]
Test Accuracy: 0.6317 | Loss: 1.3085
Class-wise Accuracy:
  Class 0: 0.2000
  Class 1: 0.7870
  Class 2: 0.5440
  Class 3: 0.4470
  Class 4: 0.4580
  Class 5: 0.6710
  Class 6: 0.8350
  Class 7: 0.7200
  Class 8: 0.8130
  Class 9: 0.8420

[Ro

In [12]:
import torch.nn.functional as F
import copy

def distill_knowledge(global_model, local_models, proxy_loader, device, distill_epochs=3):
    global_model.train()
    optimizer = torch.optim.SGD(global_model.parameters(), lr=0.01)

    for _ in range(distill_epochs):
        for images, _ in proxy_loader:
            images = images.to(device)
            ensemble_logits = torch.zeros((images.size(0), 10), device=device)

            with torch.no_grad():
                for model in local_models:
                    model.eval()
                    logits = model(images)
                    ensemble_logits += F.softmax(logits, dim=1)

            ensemble_logits /= len(local_models)
            optimizer.zero_grad()
            output = global_model(images)
            loss = F.kl_div(F.log_softmax(output, dim=1), ensemble_logits, reduction="batchmean")
            loss.backward()
            optimizer.step()

    return global_model




from torchvision import models
from torch.utils.data import DataLoader

# Init global model
global_model = models.resnet18(weights=None)
global_model.fc = nn.Linear(global_model.fc.in_features, 10)
global_model.to(device)

num_rounds = 30
num_clients = 5

for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=10,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=10,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    # FedAvg aggregation
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    # Apply knowledge distillation from round 4
    if rnd >= 10:
        proxy_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
        local_models = []
        for state in local_weights:
            local_model = models.resnet18(weights=None)
            local_model.fc = nn.Linear(local_model.fc.in_features, 10)
            local_model.to(device)
            local_model.load_state_dict(state)
            local_models.append(local_model)

        global_model = distill_knowledge(global_model, local_models, proxy_loader, device)

    # Evaluate
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    
    d["def_overall"].append(acc)
    d["def_target"].append(classwise_acc[target_class])
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")

torch.save(global_model.state_dict(), "global_model_defended_resnet18_cifar10-0.pth")
print("✅ Saved: global_model_defended_resnet18_cifar10.pth")


[Round 1]
Test Accuracy: 0.1102 | Loss: 2.4445
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0000
  Class 2: 0.0000
  Class 3: 0.0050
  Class 4: 0.0000
  Class 5: 0.9920
  Class 6: 0.0000
  Class 7: 0.0000
  Class 8: 0.1050
  Class 9: 0.0000

[Round 2]
Test Accuracy: 0.2807 | Loss: 2.0965
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0230
  Class 2: 0.2030
  Class 3: 0.5670
  Class 4: 0.0840
  Class 5: 0.6240
  Class 6: 0.3550
  Class 7: 0.0160
  Class 8: 0.5420
  Class 9: 0.3930

[Round 3]
Test Accuracy: 0.5659 | Loss: 1.3102
Class-wise Accuracy:
  Class 0: 0.1490
  Class 1: 0.6380
  Class 2: 0.3870
  Class 3: 0.4250
  Class 4: 0.3550
  Class 5: 0.6970
  Class 6: 0.7950
  Class 7: 0.5820
  Class 8: 0.8320
  Class 9: 0.7990

[Round 4]
Test Accuracy: 0.6297 | Loss: 1.3072
Class-wise Accuracy:
  Class 0: 0.1640
  Class 1: 0.7190
  Class 2: 0.5230
  Class 3: 0.4910
  Class 4: 0.5300
  Class 5: 0.6190
  Class 6: 0.8320
  Class 7: 0.6860
  Class 8: 0.8630
  Class 9: 0.8700

[Ro

In [13]:
def krum_aggregate(weight_list, f=1):
    n = len(weight_list)
    assert n > 2 * f + 2, "Not enough clients to tolerate {} Byzantine".format(f)

    flat_weights = [torch.cat([v.flatten() for v in w.values()]) for w in weight_list]
    distances = torch.zeros(n, n)
    for i in range(n):
        for j in range(i + 1, n):
            d = torch.norm(flat_weights[i] - flat_weights[j]) ** 2
            distances[i][j] = d
            distances[j][i] = d

    scores = []
    for i in range(n):
        dists = distances[i].tolist()
        dists.remove(0)
        sorted_dists = sorted(dists)
        score = sum(sorted_dists[:n - f - 2])
        scores.append(score)

    krum_index = int(np.argmin(scores))
    return copy.deepcopy(weight_list[krum_index])

global_model = models.resnet18(weights=None)
global_model.fc = nn.Linear(global_model.fc.in_features, 10)
global_model.to(device)

num_rounds = 30
num_clients = 5

# Assume train_loaders[i] and test_loader are predefined
for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:  # malicious client
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=10,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=10,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    # Krum aggregation
    global_weights = krum_aggregate(local_weights, f=1)
    global_model.load_state_dict(global_weights)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    d["krum_overall"].append(acc)
    d["krum_target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")


[Round 1]
Test Accuracy: 0.5199 | Loss: 2.7412
Class-wise Accuracy:
  Class 0: 0.7170
  Class 1: 0.5680
  Class 2: 0.0000
  Class 3: 0.5600
  Class 4: 0.4840
  Class 5: 0.0860
  Class 6: 0.5170
  Class 7: 0.7280
  Class 8: 0.8810
  Class 9: 0.6580

[Round 2]
Test Accuracy: 0.5119 | Loss: 4.0402
Class-wise Accuracy:
  Class 0: 0.8200
  Class 1: 0.6470
  Class 2: 0.0000
  Class 3: 0.5480
  Class 4: 0.3380
  Class 5: 0.0940
  Class 6: 0.5390
  Class 7: 0.7000
  Class 8: 0.8460
  Class 9: 0.5870

[Round 3]
Test Accuracy: 0.5221 | Loss: 4.2878
Class-wise Accuracy:
  Class 0: 0.8860
  Class 1: 0.6680
  Class 2: 0.0000
  Class 3: 0.4910
  Class 4: 0.3480
  Class 5: 0.0600
  Class 6: 0.5310
  Class 7: 0.8300
  Class 8: 0.8170
  Class 9: 0.5900

[Round 4]
Test Accuracy: 0.5306 | Loss: 4.0379
Class-wise Accuracy:
  Class 0: 0.8510
  Class 1: 0.6470
  Class 2: 0.0000
  Class 3: 0.5420
  Class 4: 0.4740
  Class 5: 0.0540
  Class 6: 0.4310
  Class 7: 0.8090
  Class 8: 0.8240
  Class 9: 0.6740

[Ro

In [14]:
d

{'baseline_overall': [0.1059,
  0.4396,
  0.6886,
  0.7334,
  0.7439,
  0.764,
  0.7646,
  0.7699,
  0.7736,
  0.7778,
  0.7794,
  0.7807,
  0.777,
  0.7828,
  0.7796,
  0.7788,
  0.7763,
  0.7773,
  0.7777,
  0.7808,
  0.7773,
  0.7788,
  0.779,
  0.7786,
  0.7821,
  0.7826,
  0.7828,
  0.78,
  0.7825,
  0.7798],
 'baseline_target': [0.002,
  0.622,
  0.784,
  0.804,
  0.796,
  0.824,
  0.815,
  0.82,
  0.82,
  0.839,
  0.837,
  0.835,
  0.84,
  0.821,
  0.836,
  0.853,
  0.814,
  0.826,
  0.827,
  0.83,
  0.837,
  0.843,
  0.83,
  0.823,
  0.834,
  0.824,
  0.852,
  0.84,
  0.827,
  0.83],
 'attack_overall': [0.1155,
  0.3504,
  0.5928,
  0.6317,
  0.6411,
  0.6493,
  0.6504,
  0.6501,
  0.6584,
  0.6542,
  0.6538,
  0.6575,
  0.6607,
  0.6613,
  0.6587,
  0.657,
  0.6566,
  0.6596,
  0.6567,
  0.6614,
  0.6589,
  0.6594,
  0.6613,
  0.6591,
  0.6593,
  0.6564,
  0.6583,
  0.6614,
  0.6522,
  0.6558],
 'attack_target': [0.0,
  0.001,
  0.117,
  0.2,
  0.054,
  0.025,
  0.02,
  0.016,

In [16]:
def trimmed_mean_aggregate(weight_list, n_trim):
    n_clients = len(weight_list)

    all_keys = [set(w.keys()) for w in weight_list]
    common_keys = set.intersection(*all_keys)

    aggregated_weights = {}

    for key in common_keys:
        try:
            tensors = [client[key] for client in weight_list]
            stacked = torch.stack(tensors, dim=0)

            if not torch.is_floating_point(stacked):
                stacked = stacked.float()

            sorted_vals, _ = torch.sort(stacked, dim=0)
            trimmed_vals = sorted_vals[n_trim: n_clients - n_trim]
            aggregated_weights[key] = torch.mean(trimmed_vals, dim=0)

        except Exception as e:
            print(f"Skipping key '{key}' due to error: {e}")

    return aggregated_weights


global_model = models.resnet18(weights=None)
global_model.fc = nn.Linear(global_model.fc.in_features, 10)
global_model.to(device)

num_rounds = 30
num_clients = 5

# Assume train_loaders[i] and test_loader are predefined
for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:  # malicious client
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=1,
                lr=0.001
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=1,
                lr=0.001
            )

        local_weights.append(trained_model.state_dict())

    # Krum aggregation
    global_weights = trimmed_mean_aggregate(local_weights,2)
    global_model.load_state_dict(global_weights)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    d["krum_overall"].append(acc)
    d["krum_target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")


[Round 1]
Test Accuracy: 0.1557 | Loss: 2.3181
Class-wise Accuracy:
  Class 0: 0.0020
  Class 1: 0.0550
  Class 2: 0.7750
  Class 3: 0.0010
  Class 4: 0.0000
  Class 5: 0.0710
  Class 6: 0.2650
  Class 7: 0.0040
  Class 8: 0.3320
  Class 9: 0.0520

[Round 2]
Test Accuracy: 0.2526 | Loss: 2.0431
Class-wise Accuracy:
  Class 0: 0.1140
  Class 1: 0.1660
  Class 2: 0.6050
  Class 3: 0.0080
  Class 4: 0.0000
  Class 5: 0.2110
  Class 6: 0.6230
  Class 7: 0.0310
  Class 8: 0.4910
  Class 9: 0.2770

[Round 3]
Test Accuracy: 0.3426 | Loss: 1.8098
Class-wise Accuracy:
  Class 0: 0.2500
  Class 1: 0.4330
  Class 2: 0.6090
  Class 3: 0.0100
  Class 4: 0.0000
  Class 5: 0.4170
  Class 6: 0.6600
  Class 7: 0.0710
  Class 8: 0.6640
  Class 9: 0.3120

[Round 4]
Test Accuracy: 0.4034 | Loss: 1.6219
Class-wise Accuracy:
  Class 0: 0.4520
  Class 1: 0.6330
  Class 2: 0.5540
  Class 3: 0.0240
  Class 4: 0.0030
  Class 5: 0.6300
  Class 6: 0.6240
  Class 7: 0.1790
  Class 8: 0.5280
  Class 9: 0.4070

[Ro

In [7]:
import torch
import torch.nn.functional as F
import copy
def distill_knowledge(global_model, local_models, proxy_loader, device, distill_epochs=3, temperature=3.0):
    print(f"→ Starting distillation with T = {temperature}")
    global_model.train()
    optimizer = torch.optim.SGD(global_model.parameters(), lr=0.01)

    for epoch in range(distill_epochs):
        print(f"  [Distill Epoch {epoch+1}/{distill_epochs}]")
        for images, _ in proxy_loader:
            images = images.to(device)
            ensemble_logits = torch.zeros((images.size(0), 10), device=device)

            with torch.no_grad():
                for model in local_models:
                    model.eval()
                    logits = model(images)
                    ensemble_logits += F.softmax(logits / temperature, dim=1)

            ensemble_logits /= len(local_models)
            output = global_model(images)
            student_log_probs = F.log_softmax(output / temperature, dim=1)
            loss = F.kl_div(student_log_probs, ensemble_logits, reduction="batchmean") * (temperature ** 2)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    return global_model


global_model = models.resnet18(weights=None)
device = "cuda" if torch.cuda.is_available() else "cpu"
global_model.fc = nn.Linear(global_model.fc.in_features, 10)
global_model.to(device)

num_rounds = 30
num_clients = 5

for rnd in range(num_rounds):
    print(f"\n[Round {rnd + 1}]")
    local_weights = []

    for cid in range(num_clients):
        client_model = copy.deepcopy(global_model)

        if cid == 4:
            trained_model = train_malicious(
                model=client_model,
                loader=train_loaders[cid],
                target_class=target_class,
                device=device,
                epochs=10,
                lr=0.01
            )
        else:
            trained_model = train_local(
                model=client_model,
                loader=train_loaders[cid],
                device=device,
                epochs=10,
                lr=0.01
            )

        local_weights.append(trained_model.state_dict())

    # Aggregation (FedAvg)
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    
    if rnd >= 10:
        proxy_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)
        local_models = []
        for state in local_weights:
            local_model = models.resnet18(weights=None)
            local_model.fc = nn.Linear(global_model.fc.in_features, 10)
            local_model.to(device)
            local_model.load_state_dict(state)
            local_models.append(local_model)

        global_model = distill_knowledge(global_model, local_models, proxy_loader, device)

    # Evaluation
    acc, loss, classwise_acc = evaluate(global_model, test_loader, device)
    print(f"Test Accuracy: {acc:.4f} | Loss: {loss:.4f}")
    
    d["def_overall"].append(acc)
    d["def_target"].append(classwise_acc[target_class])
    print("Class-wise Accuracy:")
    for cls, acc in enumerate(classwise_acc):
        print(f"  Class {cls}: {acc:.4f}")



[Round 1]
Test Accuracy: 0.1103 | Loss: 2.4542
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0000
  Class 2: 0.0000
  Class 3: 0.0000
  Class 4: 0.0020
  Class 5: 0.1060
  Class 6: 0.9950
  Class 7: 0.0000
  Class 8: 0.0000
  Class 9: 0.0000

[Round 2]
Test Accuracy: 0.3485 | Loss: 2.0086
Class-wise Accuracy:
  Class 0: 0.0000
  Class 1: 0.0190
  Class 2: 0.3920
  Class 3: 0.1570
  Class 4: 0.5450
  Class 5: 0.7190
  Class 6: 0.4580
  Class 7: 0.1390
  Class 8: 0.4550
  Class 9: 0.6010

[Round 3]
Test Accuracy: 0.5665 | Loss: 1.3606
Class-wise Accuracy:
  Class 0: 0.0860
  Class 1: 0.5160
  Class 2: 0.4830
  Class 3: 0.5470
  Class 4: 0.4230
  Class 5: 0.5360
  Class 6: 0.8350
  Class 7: 0.6240
  Class 8: 0.7200
  Class 9: 0.8950

[Round 4]
Test Accuracy: 0.6380 | Loss: 1.2610
Class-wise Accuracy:
  Class 0: 0.2010
  Class 1: 0.8060
  Class 2: 0.5280
  Class 3: 0.4400
  Class 4: 0.5070
  Class 5: 0.6530
  Class 6: 0.8580
  Class 7: 0.7210
  Class 8: 0.8250
  Class 9: 0.8410

[Ro